In [22]:
from angle_emb import AnglE

MALE = 0
FEMALE = 1
OTHER = 2

SEX_MAPPING = {
    "male": MALE,
    "female": FEMALE,
    "other": OTHER
}

model = AnglE.from_pretrained(
    'WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

In [36]:
import torch

def cos_sim(a, b):
    a = torch.tensor(a).to("mps")
    b = torch.tensor(b).to("mps")
    return torch.nn.functional.cosine_similarity(a, b, dim=0)

In [297]:
class Person:
    def __init__(self,
                 id: str,
                 name: str,
                 sex: int,
                 desc: str,
                 want: str,
                 pref: [int]
                 ):
        self.id = id
        self.name = name
        self.desc = desc
        self.want = want
        self.pref = pref
        self.sex = sex
        self.similarities = []

        self.embedding = model.encode([desc])[0]
        self.want_embedding = model.encode([want])[0]

    def calc_similarity(self, other: 'Person'):
        if self == other:
            return 1.0
        for sim in self.similarities:
            if sim[0] == other:
                return sim[1]

        sim = cos_sim(self.embedding, other.embedding)
        self.similarities.append(
            (other, sim)
        )
        return sim

    def sex_match(self, other: 'Person'):
        return (self.sex in other.pref) and (other.sex in self.pref)

    def get_best_match(self):
        return max(self.similarities, key=lambda x: x[1])
    
    def get_rankings(self):
        rankings = list(map(lambda x: x[0], sorted(self.similarities, key=lambda x: x[1])))

        # filter out non sex matches
        rankings = list(filter(lambda p: p.sex_match(self), rankings))

        return rankings

In [20]:
def convert_to_sexint(l: list[str]):
    return list(map(lambda x: SEX_MAPPING[x], l))

In [10]:
from pyairtable import Api
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('AIRTABLE_API_KEY')
base_id = os.getenv('AIRTABLE_BASE')

api = Api(api_key)

descrip_table = api.table(base_id, 'descs')
people_table = api.table(base_id, 'people')

In [311]:
from tqdm import tqdm

records = descrip_table.all()

people = []
for record in tqdm(records):
    fields = record['fields']
    person = people_table.get(fields['people'][0])
    person['fields']['id'] = person['id']
    person = person['fields']

    if "undesired" in person:
        print(f"skipping {person['name']}")
        continue

    people.append(
        Person(
            person['id'],
            person['name'],
            convert_to_sexint(person['sex'])[0],
            fields['desc'],
            fields['want'],
            convert_to_sexint(person['attraction'])
        )
    )

for person in tqdm(people):
    for other in people:
        if person.id != other.id:
            person.calc_similarity(other)

 82%|████████▏ | 53/65 [00:40<00:08,  1.38it/s]

skipping Vansh Gehlot


100%|██████████| 64/64 [00:07<00:00,  8.11it/s]


In [312]:
def get_all_possible(people: list[Person]):
    all_possible_matches = []

    for person in people:
        for other in people:
            if person.id != other.id and person.sex_match(other):
                all_possible_matches.append((person, other, float(person.calc_similarity(other))))

    return all_possible_matches

In [313]:
all_possible_matches = get_all_possible(people)

In [314]:
matches = []
unmatched = [x for x in people]

while len(all_possible_matches) != 0:
    all_possible_matches = sorted(all_possible_matches, key=lambda x: x[2])
    best_match = all_possible_matches.pop()

    unmatched = [x for x in unmatched if x not in best_match]

    print(f"{best_match[0].name} matched with {best_match[1].name} with a score of {best_match[2]}")
    all_possible_matches = [x for x in all_possible_matches if not (x[0] in best_match or x[1] in best_match)]
    
    print(f"Removed all matches with {best_match[0].name} and {best_match[1].name}")
    matches.append(best_match)

Jason Jiang matched with Charlotte Chiang with a score of 0.7973101735115051
Removed all matches with Jason Jiang and Charlotte Chiang
Emma Martinez  matched with Atul Thirumalai with a score of 0.7444273233413696
Removed all matches with Emma Martinez  and Atul Thirumalai
Daphne Chan matched with Owen Hsu with a score of 0.7377946376800537
Removed all matches with Daphne Chan and Owen Hsu
vidhi matched with Viraj Chhajed with a score of 0.7305639386177063
Removed all matches with vidhi and Viraj Chhajed
Lucas Choe matched with Coco with a score of 0.7119338512420654
Removed all matches with Lucas Choe and Coco
Brianna Dee matched with Dillon Price with a score of 0.6984060406684875
Removed all matches with Brianna Dee and Dillon Price
India Brar matched with Manish Yalamanchili with a score of 0.6908382177352905
Removed all matches with India Brar and Manish Yalamanchili
Ashley Rivera matched with Alyssa Corbett with a score of 0.6863813400268555
Removed all matches with Ashley Rivera

In [315]:
for match in matches:
    print(f"{match[0].name} matched with {match[1].name}")
    match_table = api.table(base_id, 'matches')
    match_table.create({
        "p1": [match[0].id],
        "p2": [match[1].id],
        "percent": match[2]
    })

print("\n\n======================")
print("Unmatched:")    

for person in unmatched:
    print(f"{person.name} was unmatched")

Jason Jiang matched with Charlotte Chiang
Emma Martinez  matched with Atul Thirumalai
Daphne Chan matched with Owen Hsu
vidhi matched with Viraj Chhajed
Lucas Choe matched with Coco
Brianna Dee matched with Dillon Price
India Brar matched with Manish Yalamanchili
Ashley Rivera matched with Alyssa Corbett
RJ Wang matched with Kimberly Cui
Madeleine matched with John Connell
swasinya jayaraman matched with Sachin Raja
Max matched with Siya Patel
Shivani Patel matched with Vaibhav Aggarwal
Melina Eftekhari matched with Jake Hekemian
Tristan Thai matched with Lila
chris s matched with Serena Kim 
Tavia Placencia matched with Anibal Anguiano
sisi  b matched with Venkat Chitturi
Mili Pavagadhi matched with Abhiram Singh 
akshay  matched with TAZ
ty matched with Hailey Chen
Jace Crocker matched with Joshua Stepter
Stevan matched with Yanelli Raygoza
Neel Redkar matched with Lily Ong
Beatrice Huang matched with Rishi Chowdary
Raayan Dhar matched with Zaara Amin
yuka matched with Divik Chotani 